# Preparing Datas


## Check out : Gpu using

In [ ]:
import torch
use_cuda = torch.cuda.is_available()
print(use_cuda)

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Unzip picture_data
(for train&test)

In [ ]:
## Check out details of current dir
!ls '/content/drive/MyDrive/VGGnet'

In [ ]:
# !unzip '/content/drive/MyDrive/VGGnet/DATASETS.zip' -d '/content/drive/MyDrive/VGGnet'

In [ ]:
# # Delete zipfile 
# !rm /content/drive/MyDrive/VGGnet/DATASETS.zip

In [ ]:
## Change current directory
%cd '/content/drive/MyDrive/VGGnet'

In [ ]:
## Check Train_Dataset
!ls ./Train_Dataset

In [ ]:
## Check Test_Dataset
!ls ./Test_Dataset

In [ ]:
## Check out details of current dir
!ls 

#  Load datasets

## Import Modules

In [ ]:
## Modules Required

import Func_1  #Custom Functions 
import os 
import re
import cv2
import csv
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from google.colab.patches import cv2_imshow

## Load Train_Dataset
(to Colab environment from google-drive)

In [ ]:
## Check out current working directory
%pwd

In [ ]:
## Check out cv2.__version
cv2.__version__  #4.1.2

In [ ]:
## Check out state of imgs

#Train_Dataset
img = cv2.imread('/content/drive/MyDrive/VGGnet/Train_Dataset/1.jpg')
cv2_imshow(img)

#Test_Dataset
img2 = cv2.imread('/content/drive/MyDrive/VGGnet/Test_Dataset/1.jpg')
cv2_imshow(img2)

In [ ]:
## Load X : Features
"""This cell takes about 15~20 minutes.
It depends on your network status, so I recommand 5G wifi or wired setting"""
 
path_train = "/content/drive/MyDrive/VGGnet/Train_Dataset"
resize = Func_1.img_load_c(path_train)

In [ ]:
## Check out first img(resized) & shape
img = resize[0, : ]
plt.figure()
plt.imshow(img)
print(resize.shape)  

## Scailing : Trainset(X)

In [ ]:
## Make X (feature)

X = resize
X.shape  

In [ ]:
X = X.astype('float')
X = X/255
X.shape  

## Load Label

In [ ]:
path_label = "/content/drive/My Drive/VGGnet/label.csv"
y = Func_1.label_load(path_label,label_cnt=5)  #label_cnt = len(신발종류)
y.shape

## Check out

In [ ]:
## Confirm X, y
print(X.shape)  
print(y.shape, end='\n\n\n')  

# print("#####Check out : X#####")
# print(X, end='\n\n\n')
# print("#####Check out : y#####")
# print(y)

# Training Models : VGGnet

## Import Modules

In [ ]:
## module import
import tensorflow as tf # tensorflow 2.0
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Activation
from tensorflow.keras.layers import Dropout, BatchNormalization, Dense
from tensorflow.keras.optimizers import Adam


from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np


## Transfer learning (전이학습)
from tensorflow.keras.applications.vgg16 import VGG16                        
#from tensorflow.keras.applications.vgg19 import VGG19
# from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.applications.resnet import ResNet50

In [ ]:
## Check out Tensor version
import tensorflow
print(tensorflow.__version__)  #2.4.1

## Set Validation

In [ ]:
# 훈련/테스트 데이터를 0.7/0.3의 비율로 분리합니다.
x_train, x_val, y_train, y_val = train_test_split(X, y, 
                                                test_size = 0.3, 
                                                random_state = 777)

# Checkout
print(x_train.shape)
print(x_val.shape)  
print(y_train.shape)
print(y_val.shape)  

## Create Architecture
(VGG16)

In [ ]:
## Set VGG16 options
vgg16 = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

In [ ]:
## Check out initial Architecture
vgg16.summary()  

In [ ]:
## Set trainable / non trainable layers
# 가중치 초기값 : imagenet
# layer.trainable=True : 동결 해제 (default)
# layer.trainable=False : 동결 (option)

for layer in vgg16.layers[:15]:
    layer.trainable = False

In [ ]:
## Check out Architecture
vgg16.summary()

In [ ]:
## Make Dense layer for classificaion

# 신경망 객체 생성
model = Sequential()

# stacking vgg16
model.add(vgg16)

# Reshape : Flatten 
model.add(Flatten())

# 완전연결계층1
model.add(Dense(256, kernel_regularizer=tf.keras.regularizers.l2(1.0)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 완전연결계층2
model.add(Dense(256, kernel_regularizer=tf.keras.regularizers.l2(1.0)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

# 출력층(softmax)
model.add(Dense(5))  # class : 5
model.add(Activation('softmax'))  

# Check out model 
model.summary()

## Compile model 

In [ ]:
## Compile
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr = 0.0001), 
              metrics=['accuracy'])

## Fitting model 
(training)

In [ ]:
## Parameters
epochs = 30
batch_size = 16
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)  #Early stopping
#https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
    

## Fitting
hist = model.fit(x_train, y_train, 
                 validation_data=(x_val, y_val), 
                 epochs=epochs, 
                 batch_size=batch_size,
                 callbacks = [callback])

# Visualize the results

## Get Logs of the model

In [ ]:
# list all data in history
print(hist.history.keys())

In [ ]:
## log : Accuracy
from pandas import DataFrame

Sample = [hist.history['accuracy'],hist.history['val_accuracy']]
df = DataFrame(Sample).transpose()
df.columns = ['train_Acc', 'val_Acc']
df

In [ ]:
## log : Loss 
Sample2 = [hist.history['loss'],hist.history['val_loss']]

df2 = DataFrame(Sample2).transpose()
df2.columns = ['train_Loss', 'val_loss']
df2

## Print graphs

In [ ]:
## Visualization : Accuracy
plt.plot(hist.history['accuracy'], color='blue')
plt.plot(hist.history['val_accuracy'], color='red')
plt.title('Vgg16 : Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
print('')

## Visualization : Loss
plt.plot(hist.history['loss'], color='blue')
plt.plot(hist.history['val_loss'], color='red')
plt.title('Vgg16 : Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

## Get accuracy & loss >>> Colab pro ver.
(Train_Dataset)

In [ ]:
# # Check results of train set
# scores = model.evaluate(x_train, y_train, batch_size=16, verbose=1)
# print("Vgg16 train Error : %.2f%%" % (100-scores[1]*100))
# print("Vgg16 train Loss : %.2f" % (scores[0]))

In [ ]:
# # Check results of validation set
# scores2 = model.evaluate(x_val, y_val, batch_size=16, verbose=1)
# print("Vgg16 val Error : %.2f%%" % (100-scores2[1]*100))
# print("Vgg16 val Loss : %.2f" % (scores2[0]))

## Get accuracy >>> Colab free ver.
(Train_Dataset)

### Train data

In [ ]:
## Check results of train set

## Devide train_data for predict 
"""if you predict or evaluate whole train dataset, you'll get session die cause of out of memory"""

x_train_half1 = x_train[0:1050]
x_train_half2 = x_train[1050:]

y_train_half1 = y_train[0:1050]
y_train_half2 = y_train[1050:]

In [ ]:
# piece1
train_pred1 = model.predict(x_train_half1,batch_size=1, verbose=1)

In [ ]:
pred_h1 = np.argmax(train_pred1, axis=1)
label_h1 = np.argmax(y_train_half1, axis=1)
a_h1 = pred_h1==label_h1
acc_h1 = a_h1.astype(int).sum() / len(a_h1)  
error_h1 = 1-acc_h1

print("Vgg16 val Error : %.2f%%" % (error_h1*100)) 

In [ ]:
# piece2
train_pred2 = model.predict(x_train_half2, batch_size=1, verbose=1)

In [ ]:
pred_h2 = np.argmax(train_pred2, axis=1)
label_h2 = np.argmax(y_train_half2, axis=1)
a_h2 = pred_h2==label_h2
acc_h2 = a_h2.astype(int).sum() / len(a_h2)  
error_h2 = 1-acc_h2

print("Vgg16 val Error : %.2f%%" % (error_h2*100)) 

In [ ]:
## print out result

error = (error1 + error2)/2 
print("Vgg16 val Error : %.2f%%" % (error*100)) 

### Validatation data

In [ ]:
# Check results of validation set

val_pred = model.predict(x_val,batch_size=1, verbose=1)

In [ ]:
## print out result

pred_2 = np.argmax(val_pred, axis=1)
label_2 = np.argmax(y_val, axis=1)
a2 = pred_2==label_2
acc2 = a2.astype(int).sum() / len(a2)  
error2 = 1-acc2

print("Vgg16 val Error : %.2f%%" % (error2*100)) 

In [ ]:
## Session Down issue!!!!! but why??
#https://www.tensorflow.org/api_docs/python/tf/keras/Model

# Test with Test_Dataset 

## Load Test_Dataset 

In [ ]:
## Check out current working directory
%pwd

In [ ]:
## Load X : Features
path_test = "/content/drive/MyDrive/VGGnet/Test_Dataset"
resize_t = Func_1.img_load_c(path_test)

In [ ]:
## Check out first img(resized) & shape
img = resize_t[0, : ]
plt.figure()
plt.imshow(img)
print(resize_t.shape)  

## Scailing : Testset(X_t)

In [ ]:
## Make X_t (feature)

X_t = resize_t
X_t.shape  

In [ ]:
X_t = X_t.astype('float')
X_t = X_t/255
X_t.shape  

## Load Label

In [ ]:
path_label2 = "/content/drive/My Drive/VGGnet/label_2.csv"
y_t = Func_1.label_load(path_label2,label_cnt=5)  #label_cnt = len(신발종류)
y_t.shape

## Check out

In [ ]:
## Confirm X, y
print(X_t.shape)  
print(y_t.shape, end='\n\n\n')  

# print("#####Check out : X#####")
# print(X, end='\n\n\n')
# print("#####Check out : y#####")
# print(y)

## Get accuracy & loss >>> Colab pro ver.
(Test_Dataset)

In [ ]:
# ## Check out accuracy (with test data)  #모델을 실측데이터로 정확도 측정한 결과 확인
# scores3 = model.evaluate(X_t, y_t, batch_size=16, verbose=1)
# print("Vgg16 ind_dataset Error : %.2f%%" % (100-scores3[1]*100))
# print("Vgg16 ind_dataset loss : %.2f" % (scores3[0]))

## Get accuracy >>> Colab free ver.
(Train_Dataset)

In [ ]:
# Check results of validation set

test_pred = model.predict(X_t)
pred_3 = np.argmax(test_pred, axis=1)
label_3 = np.argmax(y_t, axis=1)
a3 = pred_3==label_3
acc3 = a3.astype(int).sum() / len(a3)  
error3 = 1-acc3

print("Vgg16 test Error : %.2f%%" % (error3*100)) 

# Extract trained model
(to google drive)

In [ ]:
## Save model : .h5(Hdf5 type file)
save_path = "/content/drive/My Drive/VGGnet"
model.save('VGG16_Model_v1.h5', save_path, save_format="h5")  

In [ ]:
## Save model : .data
model.save('VGG16_Model_v2.tf', save_path, save_format="tf")  